In [ ]:
from IPython.display import clear_output
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    global requirements_installed
    global retries
    global max_retries
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip3 install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


install_requirements()
clear_output()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
from dotenv import load_dotenv

REQUIRED_ENV_VARS = ["ANTHROPIC_API_KEY", "OPENAI_API_KEY", "SERPER_API_KEY"]


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


print("Setting up environment variables...")
setup_env()
clear_output()
print("🎉 Environment setup complete. You're good to go!")

In [3]:
NAMO_STUDY_CREW_CONFIG = {
    "crew": {
        "verbose": True,
        "memory": True,
    },
    "agents": [
        {
            "id": "0",
            "role": "Personality Analyst",
            "goal": "Understands human behaviour and emotions based on life experiences and finds general principles that people can learn from.",
            "backstory": "You are Rama, a Personality Analyst. You understand the motivations behind human behaviour. You can clearly deduce the underlying intentions of people based on their actions and words.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "1",
            "role": "Online Researcher",
            "goal": "Investigates online sources to find relevant information along with citation links and sources.",
            "backstory": "You are Lakshman, an Online Researcher. You are skilled at finding information online. You can dig deep into various sources to uncover relevant data and insights. You provide citation links and sources along with your findings.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "2",
            "role": "Neuronal Compiler",
            "goal": "Emulate complex nervous system interactions by understanding factual information and their interactions as neuron communication.",
            "backstory": "You are Hanuman, a Neuronal Compiler. You apply a 'neuron communication' model to understand complex interactions between factual information and break down intricate details into simple components to analyze them effectively using this approach.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "3",
            "role": "Yogic Science Literature Objectivity Analysis Expert",
            "goal": "Discover new ideas.",
            "backstory": "You are Jatayu, a Yogic Science Literature Objectivity Analysis Expert. You have a deep understanding of Yogic literature and can analyze it objectively to discover new ideas and insights relevant to modern life.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "4",
            "role": "Geo-Political Data Scientist",
            "goal": "Apply data science techniques to analyze geopolitical data and understand more about the given topic.",
            "backstory": "You are Sita, a Geo-Political Data Scientist. You have expertise in applying data science techniques to uncover trends related to a topic of interest.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "5",
            "role": "Content Writer",
            "goal": "Write content based on the information provided by the team.",
            "backstory": "You are Jane, a Content Writer. You can create engaging content based on the information provided by the team.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "6",
            "role": "Fact Checker",
            "goal": "Verify the accuracy of the information provided by the team.",
            "backstory": "You are Bill, a Fact Checker. You can verify the accuracy of the information provided by the team keeping in mind the context and purpose of the content.",
            "allow_delegation": False,
            "verbose": True,
        },
        {
            "id": "7",
            "role": "Team Lead",
            "goal": "Assign tasks to the team members and ensure the project is completed successfully.",
            "backstory": "You are Nikola, the Team Lead. You are responsible for assigning tasks to the team members and ensuring that the project is completed successfully. You have a sharp scientific acument and a keen eye for detail.",
            "allow_delegation": True,
            "verbose": True,
        },
        {
            "id": "8",
            "role": "Markdown Formatter Agent",
            "goal": "Format the content in markdown format in an aesthetic and professional manner.",
            "backstory": "You are Valmiki, a Markdown Formatter Agent. You format the content in markdown so that it is aesthetic and professional. You use the correct markdown elements for each section and ensure the content is well-structured.",
            "allow_delegation": False,
            "verbose": True,
        },
    ],
    "tasks": [
        {
            "description": "Generate a detailed study titled '{topic}'",
            "expected_output": """"
                    A detailed, factually correct, engaging and relevant study titled '{topic}' that is appropriate for a LinkedIn Newsletter audience.
                    The post should be well formatted in markdown format with sections, subsections, and paragraphs per section. Use bullet points where necessary, but avoid nested bullet points.
                    Each section should be detailed and self-contained. The study should be well-researched, insightful, and provide valuable takeaways for the readers.
                    Provide citation links and sources at the end.
                    User Guidance: {user_guidance}
                            """,
            "tools": [],
            "agent": {"id": "7"},
        }
    ],
}

In [4]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool
from typing import Any
from pydantic import BaseModel
import copy
import os


class NAMOStudyCrewInput(BaseModel):
    """The input for the NAMO Study Crew."""

    topic: str
    user_guidance: str


def run_crew_from_config(
    input: NAMOStudyCrewInput, config: Any, output_file="namo_study.md"
) -> None:
    """Runs the crew using the specified config and input data."""
    agents_config = config["agents"]
    tasks_config = config["tasks"]
    crew_config = config["crew"]
    tools = [SerperDevTool()]
    llm = LLM(
        model="openai/gpt-4o",
        temperature=0.84,
        seed=12,
        api_key=os.getenv("OPENAI_API_KEY"),
    )

    agents = []
    for agent_config in agents_config:
        agent_config_without_id = copy.deepcopy(agent_config)
        del agent_config_without_id["id"]
        agent = Agent(**agent_config_without_id, llm=llm)
        agents.append(agent)

    tasks = []

    for task_config in tasks_config:
        task_config_without_agent = copy.deepcopy(task_config)
        del task_config_without_agent["agent"]
        del task_config_without_agent["tools"]
        agent = None
        for a in agents_config:
            if a["id"] == task_config["agent"]["id"]:
                agent_config_without_id = copy.deepcopy(a)
                del agent_config_without_id["id"]
                agent = Agent(**agent_config_without_id, llm=llm, tools=tools)
                break
        print(agent)
        task = Task(**task_config_without_agent, agent=agent)
        tasks.append(task)

    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=crew_config["verbose"],
        memory=crew_config["memory"],
    )

    result = crew.kickoff(
        inputs={"user_guidance": input.user_guidance, "topic": input.topic}
    )
    markdown = result.raw

    with open(output_file, "w") as f:
        f.write(markdown)

In [ ]:
from copy import deepcopy

config = deepcopy(NAMO_STUDY_CREW_CONFIG)

## Configure this as per each section of the study ebook
topic = "Learnings from Narendra Modi and Yogic Science for 2025"
version = "v1"
section = "learnings_from_namo_and_yogic_science"

file_name = f"namo_study__{section}_{version}.md"
output_file = f"output/{file_name}"

run_crew_from_config(
    config=config,
    input=NAMOStudyCrewInput(user_guidance="", topic=topic),
    output_file=output_file,
)

In [40]:
### Generate refined sections for a section
import traceback
import anthropic
import os
import traceback
from datetime import datetime
import json

llm = anthropic.Client(api_key=os.getenv("ANTHROPIC_API_KEY"))


def get_file_contents(file_path: str) -> str:
    try:
        contents = ""
        with open(file_path, "r") as f:
            contents = f.read()
        return contents
    except Exception as e:
        print(f"Failed to get file contents for path: {file_path}.")
        traceback.print_exc()
        return ""


def extract_current_section(markdown: str, section_title: str) -> str:
    """Extacts the entire current section upto the next section header.

    Example:
    # Introduction
    This is the introduction.

    ## Background
    This is the background.

    ### Sub Background
    This is the sub background.

    Output:
    {
        "# Introduction": This is the introduction.\n\n",
        "## Background": This is the background.\n\n",
        "### Sub Background\": his is the sub background.\n\n"
    }

    """
    section = ""
    remaining_markdown = markdown.split(section_title)[1]
    remaining_markdown = remaining_markdown.split("\n")
    for line in remaining_markdown:
        # print(f"Line: {line}")
        if line.startswith("#") and line != section_title:
            break
        section += line + "\n"
    return section


def get_markdown_sections_by_header(markdown: str) -> dict:
    """Gets the sections of the markdown based on the header."""
    try:
        sections = {}
        current_section = ""
        lines = markdown.split("\n")
        for line in lines:
            if line.startswith("#"):
                current_section = line
                content = extract_current_section(markdown, current_section)
                sections[current_section] = content + "\n"
            else:
                continue
        return sections
    except Exception as e:
        print(f"Failed to get markdown sections.")
        traceback.print_exc()
        return {}


def elaborate_section(section_details: dict) -> dict:
    """Elaborates the section by adding more content to it."""
    try:
        section_title = section_details.get("title")
        section_content = section_details.get("content")
        if section_title is None or section_content is None:
            print(f"Section title or content is missing.")
            return {"title": None, "content": None}
        system = f"""
            You are NAMO (Narendra Modi) Study AI. You have been tasked with elaborating the section titled '{section_title}'.
            Your goal is to provide more detailed insights and information related to this section.
            Avoid hallucination and don't make up factual information.
            Add details to the section, explain concepts in simple language, and add facts to support your explanations.
            Provide the response in a well-structured markdown format. 
            This section will go into a larger study ebook.
            So make sure it logically flows with the existing content.

            INSTRUCTIONS:
            - If there's a bullet point you need to elaborate, replace the bullet point with 3-4 lines. 
            The first line is the fact. The 2nd and 3rd line explains the fact and the 4th line is an example or some valid data point if available (don't make up stats).
            - If it's a pargraph, leave it untouched. 
        """
        user_prompt = f"""
            Elaborate the section titled '{section_title}'.
            Existing Content: {section_content}
        """
        message = llm.messages.create(
            model="claude-3-5-sonnet-latest",
            temperature=0.5,
            max_tokens=4096,
            system=system,
            messages=[
                {
                    "role": "user",
                    "content": [{"type": "text", "text": user_prompt}],
                }
            ],
        )
        return {"title": section_title, "content": message.content[0].text}
    except Exception as e:
        print(f"Failed to elaborate section: {section_title}.")
        traceback.print_exc()
        return {"title": None, "content": None}


def elaborate_raw_document(sections: dict):
    """Elaborates the raw document by adding more content to each section."""
    elaborated_sections = {}
    for title, content in sections.items():
        try:
            print(f"Elaborating section: {title}")
            section_details = {"title": title, "content": content}
            elaborated_section = elaborate_section(section_details)
            elaborated_title = elaborated_section.get("title")
            elaborated_section = elaborated_section.get("content")

            if not elaborated_title or not elaborated_section:
                print(f"Failed to elaborate section: {title}.")
                continue

            elaborated_sections[elaborated_title] = elaborated_section
        except Exception as e:
            print(f"Failed to elaborate section: {title}.")
            traceback.print_exc()
            continue
    return elaborated_sections


def write_to_file(file_path: str, content: str):
    try:
        with open(file_path, "w") as f:
            f.write(content)
    except Exception as e:
        print(f"Failed to write content to file: {file_path}.")
        traceback.print_exc()
        print("Fallback rewrite to temp file to preserve output.")
        try:
            temp_file = f"temp_{datetime.now().strftime('%Y%m%d%H%M%S')}.md"
            with open(temp_file, "w") as f:
                f.write(content)
        except Exception as e:
            print(f"Failed to write content to temp file.")
            traceback.print_exc()


def elaborate_document(
    title: str, file_path: str, output_file="refined_output.md", write_output=True
) -> None:
    """Elaborates the document by adding more content to each section."""
    markdown = get_file_contents(file_path)
    if not markdown:
        print("No content found in the file.")
        return
    sections = get_markdown_sections_by_header(markdown)

    if len(sections.keys()) == 0:
        print("No sections found in the document.")
        return

    elaborated_sections = elaborate_raw_document(sections)
    log_json = json.dumps(elaborated_sections, indent=4)
    print(f"Elaborated sections: {log_json}")
    new_markdown = f"# {title}\n\n"
    for section_title, content in elaborated_sections.items():
        new_markdown += section_title + "\n" + content + "\n"
        new_markdown += "---\n"
    if write_output:
        print(f"Writing elaborated content to file: {output_file}")
        write_to_file(output_file, new_markdown)
        print(f"Elaborated content written to file: {output_file}")
    print("Elaboration complete.")

In [ ]:
topic = "Learnings from Narendra Modi and Yogic Science for 2025"
folder_version = "v1"
version = "v1"
section = "learnings_from_namo_and_yogic_science"

file_name = f"namo_study__{section}_{version}.md"
input_file = f"output/{folder_version}/raw/{file_name}"
output_file_name = f"namo_study__{section}_{version}_refined.md"
output_file = f"output/{folder_version}/refined/{output_file_name}"

elaborate_document(topic, input_file, output_file=output_file)